In [ ]:
!pip install transformers datasets scikit-learn
!pip install hf_xet

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Jedzonko.xlsx to Jedzonko (2).xlsx


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os
from typing import List, Dict  # Dodano typowanie

In [ ]:
try:
    df = pd.read_excel("Jedzonko.xlsx")
    print("Plik załadowano pomyślnie!")
    print(df.head()) # Opcjonalnie, wyświetl kilka pierwszych wierszy
except FileNotFoundError:
    print("Błąd: Nie znaleziono pliku 'Jedzonko.xlsx'. Upewnij się, że został załadowany.")
    exit()

Plik załadowano pomyślnie!
   Column1                                              Title  \
0        0  Miso-Butter Roast Chicken With Acorn Squash Pa...   
1        1                    Crispy Salt and Pepper Potatoes   
2        2                        Thanksgiving Mac and Cheese   
3        3                 Italian Sausage and Bread Stuffing   
4        4                                       Newton's Law   

                                        Instructions  \
0  Pat chicken dry with paper towels, season all ...   
1  Preheat oven to 400°F and line a rimmed baking...   
2  Place a rack in middle of oven; preheat to 400...   
3  Preheat oven to 350°F with rack in middle. Gen...   
4  Stir together brown sugar and hot water in a c...   

                                          Image_Name  \
0  miso-butter-roast-chicken-acorn-squash-panzanella   
1         crispy-salt-and-pepper-potatoes-dan-kluger   
2         thanksgiving-mac-and-cheese-erick-williams   
3          italian-sa

In [ ]:
os.environ["WANDB_DISABLED"] = "true"  # disable wandb logging

In [ ]:
MODEL_NAME = "t5-base"  # Zmieniono na większy model
OUTPUT_DIR = "./recipe_model"
LOGGING_DIR = "./logs"
MAX_INPUT_LENGTH = 128
MAX_OUTPUT_LENGTH = 512
TRAIN_BATCH_SIZE = 4
NUM_TRAIN_EPOCHS = 2  # Zwiększono liczbę epok

In [ ]:
def clean_ingredients(ingredient_string: str) -> str:
    """
    Czyści listę składników, usuwając nadmiarowe znaki, małe elementy i duplikaty.

    Args:
        ingredient_string: Ciąg tekstowy zawierający listę składników.

    Returns:
        Ciąg tekstowy z wyczyszczoną listą składników.
    """

    items = re.split(r',|\n', str(ingredient_string))
    cleaned = []
    seen = set()

    for item in items:
        item = item.strip().lower()
        # Rozszerzono warunek czyszczenia
        if len(item) > 3 and item not in seen and not re.search(r'\d+\s*(g|kg|ml|l|cup|tablespoon|teaspoon|oz)', item):
            # Pomijaj składniki, które wyglądają jak ilości (np. "10g cukru")
            cleaned.append(item)
            seen.add(item)

    return "\n".join(cleaned)

In [ ]:
def clean_instructions(text: str) -> str:
    """
    Czyści instrukcje, usuwając nadmiarowe powtórzenia i formatowanie.

    Args:
        text: Ciąg tekstowy zawierający instrukcje.

    Returns:
        Ciąg tekstowy z wyczyszczonymi instrukcjami.
    """

    text = re.sub(r'(sugar\s*,?)+', 'sugar, ', text, flags=re.IGNORECASE)
    text = re.sub(r'(sage\s*,?)+', 'sage, ', text, flags=re.IGNORECASE)
    text = re.sub(r'(\s*,\s*)+', ', ', text)
    text = re.sub(r'(add|stir in|mix in|combine) (sugar|sage)( and \2)+', r'\1 \2', text, flags=re.IGNORECASE)
    text = re.sub(r'\.+', '.', text)  # Usuń wielokrotne kropki
    text = re.sub(r'[\n\t]+', ' ', text)  # Usuń nowe linie i tabulatory
    return text.strip()

In [ ]:
def prepare_data(row: pd.Series) -> Dict[str, str]:
    """
    Przygotowuje dane do treningu, łącząc tytuł, składniki i instrukcje.

    Args:
        row: Wiersz DataFrame zawierający dane przepisu.

    Returns:
        Słownik zawierający sformatowane dane wejściowe i wyjściowe.
    """

    input_text = f"generate recipe: {row['Title']}"
    cleaned_ingredients = clean_ingredients(row['Cleaned_Ingredients'])
    cleaned_instructions = clean_instructions(str(row['Instructions']))
    output_text = f"Składniki:\n{cleaned_ingredients}\n\nInstrukcje:\n{cleaned_instructions}"
    return {"input": input_text, "output": output_text}

In [ ]:
def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Przetwarza DataFrame, przygotowując dane do treningu.

    Args:
        df: DataFrame zawierający dane przepisów.

    Returns:
        DataFrame z kolumnami 'input' i 'output' gotowymi do treningu.
    """
    df = df.dropna(subset=['Title', 'Cleaned_Ingredients', 'Instructions'])
    data = df.apply(prepare_data, axis=1, result_type='expand')
    data.columns = ['input', 'output']
    return data

In [ ]:
def create_datasets(data: pd.DataFrame) -> (Dataset, Dataset):
    """
    Dzieli dane na zbiory treningowe i testowe.

    Args:
        data: DataFrame z danymi.

    Returns:
        Tuple zawierający zbiory treningowe i testowe jako obiekty Dataset.
    """
    train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)  # Dodano random_state
    train_dataset = Dataset.from_pandas(train_data)
    test_dataset = Dataset.from_pandas(test_data)
    return train_dataset, test_dataset


In [ ]:
def preprocess_function(examples: Dict[str, List]) -> Dict[str, List[List[int]]]:
    """
    Tokenizuje dane wejściowe i wyjściowe.

    Args:
        examples: Słownik zawierający listy tekstów wejściowych i wyjściowych.

    Returns:
        Słownik zawierający tokenizowane dane.
    """
    inputs = tokenizer(examples['input'], padding="max_length", truncation=True, max_length=MAX_INPUT_LENGTH)
    labels = tokenizer(examples['output'], padding="max_length", truncation=True, max_length=MAX_OUTPUT_LENGTH)
    inputs["labels"] = labels["input_ids"]
    return inputs


In [ ]:
def train_model(train_dataset: Dataset, test_dataset: Dataset) -> T5ForConditionalGeneration:
    """
    Trenuje model T5.

    Args:
        train_dataset: Zbiór danych treningowych.
        test_dataset: Zbiór danych testowych.

    Returns:
        Wytrenowany model T5.
    """

    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=TRAIN_BATCH_SIZE,
        num_train_epochs=NUM_TRAIN_EPOCHS,
        save_strategy="epoch",
        logging_dir=LOGGING_DIR,
        logging_steps=10,
        fp16=torch.cuda.is_available(),
        learning_rate=2e-4,  # Dodano learning rate
        weight_decay=0.01,  # Dodano weight decay
        gradient_accumulation_steps=2,  # Dodano akumulację gradientu
        # evaluation_strategy="epoch", # Włącz ewaluację w trakcie treningu (opcjonalne)
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset  # Dodaj test_dataset do ewaluacji (opcjonalne)
    )

    trainer.train()
    return model

In [ ]:
def generate_recipe(model: T5ForConditionalGeneration, tokenizer: T5Tokenizer, input_text: str) -> Dict[str, List[str]]:
    """
    Generuje przepis na podstawie podanego tekstu.

    Args:
        model: Wytrenowany model T5.
        tokenizer: Tokenizer T5.
        input_text: Tekst wejściowy, np. "pizza".

    Returns:
        Słownik zawierający listę składników i instrukcje.
    """

    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids,
        max_length=MAX_OUTPUT_LENGTH,
        num_beams=8,  # Zwiększono liczbę belek
        no_repeat_ngram_size=3,
        early_stopping=True,
        temperature=0.8,  # Zwiększono temperaturę
        do_sample=True,  # Włączono sampling
        top_k=50,  # Dodano top-k sampling
        top_p=0.95  # Dodano top-p sampling
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    parts = decoded_output.split("Instrukcje:")
    ingredients = parts[0].replace("Składniki:", "").strip().split("\n")

    # Sprawdzanie i czyszczenie instrukcji
    if len(parts) > 1:
        instructions = parts[1].strip()
        instructions = re.sub(r'[\n\t]+', ' ', instructions)  # Dodatkowe czyszczenie instrukcji
    else:
        instructions = "Brak instrukcji."

    # Czyszczenie składników
    ingredients = [ingredient.strip() for ingredient in ingredients if ingredient.strip()]
    ingredients = list(dict.fromkeys(ingredients))  # Usuwanie duplikatów (zachowuje kolejność)

    return {"Składniki": ingredients, "Instrukcje": instructions}


In [ ]:
if __name__ == '__main__':
    import os
    from google.colab import files

    # Przygotowanie danych
    try:
        df = pd.read_excel("Jedzonko.xlsx")  # Zmień na nazwę swojego pliku
        print("Plik załadowano pomyślnie!")
    except FileNotFoundError:
        print("Błąd: Nie znaleziono pliku. Upewnij się, że plik istnieje i został załadowany.")
        exit()

    # Przetwarzanie danych
    data = process_dataframe(df)
    train_dataset, test_dataset = create_datasets(data)

    # Inicjalizacja modelu i tokenizera
    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
    model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to("cuda" if torch.cuda.is_available() else "cpu")

    # Tokenizacja danych
    tokenized_train = train_dataset.map(preprocess_function, batched=True)
    tokenized_test = test_dataset.map(preprocess_function, batched=True)

    # Trening modelu
    trained_model = train_model(tokenized_train, tokenized_test)

    # Zapis i pobranie modelu
    if trained_model is not None:
        trained_model.save_pretrained("./recipe_model")
        tokenizer.save_pretrained("./recipe_model")

        # Spakowanie do ZIP
        os.system('zip -r recipe_model.zip ./recipe_model')

        # Pobranie ZIP na komputer (działa tylko w Colab)
        files.download("recipe_model.zip")

        print("Model i tokenizer zostały zapisane i pobrane jako recipe_model.zip")
    else:
        print("Trening modelu nie został zakończony, więc nie można zapisać modelu.")

In [ ]:
# Generowanie przykładowego przepisu
    example_input = "Spanakopita"
    recipe = generate_recipe(trained_model, tokenizer, example_input)
    print(f"Przepis dla: {example_input}")
    print(recipe)

{'Składniki': ["Skadniki: '1 pound Spanakopita' 'kosher salt freshly ground pepper']"], 'Instrukcje': 'Stir together lime juice, lemon juice, and lime juice in a medium bowl. Season with salt and pepper, then serve with lime wedges garnished with lemon wedges.'}


In [ ]:
from google.colab import files
files.download("recipe_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
uploaded = files.upload()
!unzip recipe-model.zip
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

KeyboardInterrupt: 

In [ ]:
model_path = "./recipe_model"  # adjust path as needed
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)